<a href="https://colab.research.google.com/github/almunawaroh2020-maker/Drug-discovery-AI-course-2026/blob/main/Assignment_2_qsar_data_curation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI And Biotechnology/Bioinformatics**

## **AI and Drug Discovery Course: QSAR Modeling**
This notebook demonstrates how to collect and preprocess bioactivity data from ChEMBL for QSAR modeling

# **Part 1: Data Collection & Curation**

**First we need to connect Google Colab with our Google Drive, so that we can have access to our Google drive within Colab.**

This allows us to:
* Save datasets
* Reload data across sessions
* Organize project files




In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

**Now create "data" folder in our "Colab Notebooks" folder on Google Drive.**

In [ ]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/data"

## Install and Import Required Libraries
We install the ChEMBL web service package so that we can retrieve bioactivity data

In [ ]:
!pip install chembl_webresource_client

# Import Libraries
* pandas for data handling
* new_client from chembl for accessing the database

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

# Step 1: Search for Traget Protein

## **Target Identification (KRAS)**
Search ChEMBL for the KRAS target and select the most relevant entry.


In [ ]:
target = new_client.target
target_query = target.search("KRAS")
targets = pd.DataFrame.from_dict(target_query)
targets.head()


**Reterive Bioactivity data for selected target**

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target

**Now retrieve only bioactivity data for target; **GTPase KRas(CHEMBL2189121)** with reported IC 50  values in nM (nanomolar) unit.**

In [ ]:
activity = new_client.activity
results = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df1 = pd.DataFrame.from_dict(results)
df1.head(5)

In [ ]:
df1.standard_type.unique()

**Finally Save the resulting bioactivity data to a CSV file** **bioactivity_raw_data.csv**.

In [ ]:
df1.to_csv('bioactivity_raw_data.csv', index=False)

**Now copy "bioactivity_raw_data.csv" file to Google Drive, in foler "data"**

In [ ]:
! cp bioactivity_raw_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

In [ ]:
! ls -l "/content/gdrive/My Drive/Colab Notebooks/data"

In [ ]:
! head bioactivity_raw_data.csv

# **Step 3: Bioactivity Data Retrieval (IC50)**
**Retrieve bioactivity data (IC50) for the selected KRAS target.**

**Inspect Missing Values**

In [ ]:
df1["standard_type"].isna().sum()

**Filter Rows with Valid Bioactivity Values**

In [ ]:
df2 = df[df["standard_value"].notna()]
df2.head()

**Assign Bioactivity Classes**
Define active, intermediate, and inactive classes based on IC50 values.


In [ ]:
bioactivity_class = []
for value in df2.standard_value:
    value = float(value)
    if value >= 10000:
        bioactivity_class.append("inactive")
    elif value <= 1000:
        bioactivity_class.append("active")
    else:
        bioactivity_class.append("intermediate")

**Extract Relevant Columns**

In [ ]:
molecule_ids = df2.molecule_chembl_id.tolist()
canonical_smiles = df2.canonical_smiles.tolist()
standard_values = df2.standard_value.tolist()

In [ ]:
data = list(zip(
    molecule_ids,
    canonical_smiles,
    standard_values,
        bioactivity_class,
))

**Create Preprocessed bioactivity Dataset**

In [ ]:

df3 = pd.DataFrame(
    data,
    columns=[
        "molecule_chembl_id",
        "canonical_smiles",
        "standard_value",
        "bioactivity_class",
    ]
)
df3.head()

**Remove Compounds without Valid SMILES**. Drop rows with **NaN**, **empty** or **None** SMILES values.

In [ ]:
df3 = df3.dropna(subset=["canonical_smiles"])
df3 = df3[df3["canonical_smiles"].str.lower() != "none"]
df3 = df3[df3["canonical_smiles"].str.strip() != ""]
df3.head()

**Save Preprocessed Bioactivity Data.** Save the cleaned dataset to CSV and copy to Google Drive.

In [ ]:
df3.to_csv("bioactivity_preprocessed_data.csv", index=False)

!cp bioactivity_preprocessed_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"
!ls "/content/gdrive/My Drive/Colab Notebooks/data"

## **End of Part 1: Data Collection and Curation**